In [26]:
import pandas as pd

#####THESE NEEDS TO BECOME FUNCTION INPUTS####
min_carb_intake, max_carb_intake = 250, 300

food_path = 'MyFoodData.csv'
#############################################

def meal_recommendation(fridge):
    TARGET_AVERAGE_RANK = 8
    if min_carb_intake < 0 or min_carb_intake > 300:
        raise ValueError(f"Invalid min carbohydrate intake: {min_carb_intake}")
    if max_carb_intake < 0 or max_carb_intake > 300 or max_carb_intake < min_carb_intake:
        raise ValueError(f"Invalid max carbohydrate intake: {max_carb_intake}")
    menu = pd.DataFrame()
    menu = menu_builder(menu,fridge)
    return menu

#############################################

def prepare_fridge(food_path):
    FRIDGE_ITEMS = 30
    food_data = pd.read_csv(food_path, header=0)
    fridge = food_data.sample(n=FRIDGE_ITEMS, replace=False, ignore_index=True)
    empty_fridge = pd.DataFrame(columns=list(fridge), index=[0])
    fridge = pd.concat([empty_fridge,fridge], ignore_index=True)
    return fridge

def check_valid_menu(menu):
    proposed_menu = menu.drop_duplicates()
    total_carbs_check = min_carb_intake <= proposed_menu.loc[:,'Carbohydrate'].sum() <= max_carb_intake
    total_item_check = 2 <= proposed_menu.loc[:,'Carbohydrate'].count() <= 4
    average_rank_check = proposed_menu.loc[:,'Ranking'].mean() >= 8
    contains_beverage = not proposed_menu[proposed_menu.isin(['Beverages'])].dropna(how='all').empty
    return (total_carbs_check and total_item_check and
        average_rank_check and contains_beverage)
        

In [3]:
fridge = prepare_fridge(food_path)
# fridge.loc[:,'Carbohydrate'].sum()
# fridge.loc[:,'Carbohydrate'].count() check that are are at least 2 items and less than 4
# fridge.loc[:,'Ranking'].mean()
# fridge.loc[:,'ID'].nunique() #check the number of unique IDs in the dataframe
# fridge.loc[:,'ID'].count() # check the number of IDs to compare to unique to find duplicates
# fridge.drop_duplicates()
# fridge[fridge.isin(['Beverages'])].dropna(how='all').empty # check that there at least one beverage 

In [4]:
menu = pd.DataFrame()
def menu_builder(menu,fridge): 
    for i in range(fridge.shape[0]):
        menu = pd.concat([menu,fridge.iloc[[i]]], ignore_index = True)
        for i2 in range(fridge.shape[0]):
            menu = pd.concat([menu,fridge.iloc[[i2]]], ignore_index = True)
            for i3 in range(fridge.shape[0]):
                menu = pd.concat([menu,fridge.iloc[[i3]]], ignore_index = True)
                for i4 in range(fridge.shape[0]):
                    menu = pd.concat([menu,fridge.iloc[[i4]]], ignore_index = True)
                    if check_valid_menu(menu):
                        menu.dropna(how = 'all', inplace = True)
                        return menu
                    menu.drop([3], inplace = True)
                menu.drop([2], inplace = True)
            menu.drop([1], inplace = True)
        menu.drop([0], inplace = True)
    else:
        return "No valid Menu"

In [8]:
fridge

,ID,Name,Food Group,Carbohydrate,Ranking
0,NaN,NaN,NaN,NaN,NaN
1,168122,Coffee Instant With Whitener Reduced Calorie,Beverages,60,6
2,174385,Lamb New Zealand Imported Frozen Composite Of ...,Meats,0,3
3,174716,Beef New Zealand Imported Bolar Blade Separabl...,Meats,0,3
4,168574,Fava Beans (Raw),Beans and Lentils,18,5
5,170210,Beef Brisket Flat Half Separable Lean Only Tri...,Meats,0,4
6,787440,Turnip Greens Canned Reduced Sodium Cooked Ns ...,Vegetables,3,6
7,167613,Owl Horned Flesh Raw (Alaska Native),American Indian,0,2
8,172911,Soup Chili Beef Canned Prepared With Equal Vol...,Soups and Sauces,9,2
9,173949,Wild Blueberries (Frozen),Fruits,14,10


In [ ]:
meal_recommendation(fridge)